# Exemplo de self-attention

In [6]:
import torch

# Define um tensor `inputs` com 6 vetores, cada um com 3 dimensões.
# Cada vetor pode ser interpretado como a representação (embedding) de uma palavra/token.
inputs = torch.tensor(
  [[0.43, 0.15, 0.89],
   [0.55, 0.87, 0.66],  # <- Este será usado como "query"
   [0.57, 0.85, 0.64],
   [0.22, 0.58, 0.33],
   [0.77, 0.25, 0.10],
   [0.05, 0.80, 0.55],]
)

# Seleciona o segundo vetor como `query`, ou seja, a palavra que está consultando o contexto.
query = inputs[1]

# Cria um vetor vazio `attn_scores_2` para armazenar os scores de atenção.
# O tamanho é igual ao número de vetores de entrada.
attn_scores_2 = torch.empty(inputs.shape[0])

# Calcula o score de atenção entre o `query` e cada vetor de entrada (inclusive ele mesmo).
# Neste caso, o score é apenas o produto escalar (dot product), que mede similaridade.
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

# Exibe os scores de atenção calculados
print(attn_scores_2)


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


* A soma dos pesos é 1

In [7]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


## Normalização

In [8]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


* Normalização correta 

In [9]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [10]:
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


# Computando os pesos da atenção para todos os inputs 

In [11]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [12]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [13]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [14]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [15]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [16]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


# Self-Attention

In [17]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

In [18]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [19]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


In [20]:
keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [21]:
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [22]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1476, 0.2164, 0.2134, 0.1365, 0.1240, 0.1621])


In [23]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.2991, 0.8010])


In [26]:
import torch.nn as nn

# Define uma classe de atenção própria (Self-Attention)
class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()

        # Matrizes de projeção para query, key e value
        # São parâmetros treináveis com dimensões (d_in, d_out)
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        # Calcula as queries, keys e values aplicando as projeções lineares
        # Aqui usamos multiplicação de matriz diretamente: (batch, d_in) @ (d_in, d_out)
        keys = self.W_key(x)      # shape: (seq_len, d_out)
        queries = self.W_query(x)    # shape: (seq_len, d_out)
        values = self.W_value(x)    # shape: (seq_len, d_out)
        
        # Calcula os "attention scores" fazendo o produto escalar entre queries e transposta de keys
        # shape: (seq_len, d_out) @ (d_out, seq_len) → (seq_len, seq_len)
        attn_scores = queries @ keys.T  # matriz de similaridade entre todos os pares de tokens

        # Normaliza os scores com softmax (com escalamento pela raiz da dimensão dos keys)
        # Isso transforma os scores em pesos de atenção
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        # Multiplica os pesos de atenção pelos valores para obter o vetor de contexto
        # shape: (seq_len, seq_len) @ (seq_len, d_out) → (seq_len, d_out)
        context_vec = attn_weights @ values

        # Retorna os vetores de contexto (um para cada entrada)
        return context_vec

# Define uma semente para garantir reprodutibilidade
torch.manual_seed(123)

# Cria uma instância da atenção com as dimensões de entrada e saída (assumidos definidos antes)
sa_v1 = SelfAttention_v1(d_in, d_out)

# Aplica a camada de self-attention aos dados de entrada 'inputs'
print(sa_v1(inputs))


tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)


In [28]:
queries = sa_v1.W_query(inputs)
keys = sa_v1.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1717, 0.1762, 0.1761, 0.1555, 0.1627, 0.1579],
        [0.1636, 0.1749, 0.1746, 0.1612, 0.1605, 0.1652],
        [0.1637, 0.1749, 0.1746, 0.1611, 0.1606, 0.1651],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.1632, 0.1674],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.1639],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)


In [29]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [30]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1717, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1749, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1637, 0.1749, 0.1746, 0.0000, 0.0000, 0.0000],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.0000, 0.0000],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<MulBackward0>)


In [31]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<DivBackward0>)


In [32]:
# Cria uma máscara triangular superior (acima da diagonal principal) de tamanho (context_length x context_length)
# torch.ones(...) cria uma matriz cheia de 1s
# torch.triu(...) zera todos os elementos abaixo da diagonal (inclusive a diagonal se diagonal=1)
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)

# Aplica a máscara aos scores de atenção (attn_scores)
# Substitui (com masked_fill) os valores onde a máscara for True por -∞ (menos infinito)
# Isso é feito para impedir que um token "veja" tokens futuros — típica máscara usada em modelos de linguagem autoregressivos como o GPT
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)

# Exibe os scores de atenção já com a máscara aplicada
print(masked)


tensor([[0.3111,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1655, 0.2602,   -inf,   -inf,   -inf,   -inf],
        [0.1667, 0.2602, 0.2577,   -inf,   -inf,   -inf],
        [0.0510, 0.1080, 0.1064, 0.0643,   -inf,   -inf],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121,   -inf],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MaskedFillBackward0>)


In [33]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)


# Dropout

In [34]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [35]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0335, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6804, 0.0000, 0.0000, 0.0000],
        [0.4889, 0.5090, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3988, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3418, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


In [36]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [37]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()

        self.d_out = d_out

        # Projeções lineares para obter queries, keys e values a partir do input x
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        # Dropout aplicado nos pesos de atenção para regularização
        self.dropout = nn.Dropout(dropout)

        # Máscara triangular superior usada para bloquear posições futuras (atenção causal)
        # Register_buffer mantém o tensor como parte do estado do módulo sem ser um parâmetro treinável
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        # x tem shape [batch_size, num_tokens, d_in]
        b, num_tokens, d_in = x.shape

        # Projeções lineares
        keys = self.W_key(x)      # shape: [b, num_tokens, d_out]
        queries = self.W_query(x) # shape: [b, num_tokens, d_out]
        values = self.W_value(x)  # shape: [b, num_tokens, d_out]

        # Calcula pontuações de atenção com produto escalar entre queries e transposição de keys
        attn_scores = queries @ keys.transpose(1, 2)  # shape: [b, num_tokens, num_tokens]

        # Aplica a máscara causal para evitar que posições acessem tokens futuros
        # A máscara define -inf para as posições superiores à diagonal
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )

        # Normaliza as pontuações com softmax (dividido pela raiz de d_k para estabilidade)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        # Aplica dropout nos pesos de atenção
        attn_weights = self.dropout(attn_weights)

        # Combina os valores com os pesos de atenção para obter os vetores de contexto
        context_vec = attn_weights @ values  # shape: [b, num_tokens, d_out]
        return context_vec


# Define semente para reprodução
torch.manual_seed(123)

# Usa o número de tokens como comprimento de contexto
context_length = batch.shape[1]

# Cria a instância da atenção causal
ca = CausalAttention(d_in, d_out, context_length, dropout=0.0)

# Aplica a atenção causal sobre o batch
context_vecs = ca(batch)

# Exibe o resultado final
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)  # Deve ser [2, 6, d_out]


tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


# Multi-head attention

In [38]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()

        # Cria uma lista de cabeças de atenção (CausalAttention)
        # Cada cabeça é independente e aprende diferentes representações
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        # Executa cada cabeça de atenção e concatena os resultados ao longo da última dimensão (d_out)
        return torch.cat([head(x) for head in self.heads], dim=-1)


# Define a semente aleatória para reprodutibilidade
torch.manual_seed(123)

# Define o comprimento de contexto como o número de tokens no batch
context_length = batch.shape[1]  # Ex: 6 tokens por sequência

# Define os tamanhos de entrada e saída da projeção
d_in, d_out = 3, 2  # Cada token tem dimensão 3, saída de cada cabeça será 2

# Cria uma instância do wrapper de atenção multi-cabeça com 2 cabeças
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, dropout=0.0, num_heads=2
)

# Aplica a atenção multi-cabeça ao batch de entrada
context_vecs = mha(batch)

# Mostra os vetores de contexto resultantes
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)  # Esperado: [batch_size, num_tokens, num_heads * d_out] = [2, 6, 4]


tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


* Com pesos

In [43]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        
        # Garante que a dimensão de saída possa ser dividida igualmente entre as cabeças
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Dimensão interna de cada cabeça (porção do d_out)

        # Projeções lineares para Q, K, V com dimensão de saída igual a d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        # Camada linear final para projetar a concatenação das cabeças de volta para d_out
        self.out_proj = nn.Linear(d_out, d_out)

        self.dropout = nn.Dropout(dropout)

        # Máscara causal (superior triangular) para impedir que tokens "vejam o futuro"
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        # Projeta as entradas em Q, K e V de dimensão (b, num_tokens, d_out)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # Divide d_out em múltiplas cabeças: (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpõe para que a dimensão das cabeças venha antes dos tokens:
        # (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Produto escalar entre Q e K^T: (b, num_heads, num_tokens, num_tokens)
        attn_scores = queries @ keys.transpose(2, 3)

        # Aplica a máscara causal para impedir atenção ao futuro
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        # Normaliza as pontuações com softmax e aplica dropout
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Multiplica pesos de atenção por V: (b, num_heads, num_tokens, head_dim)
        context_vec = attn_weights @ values

        # Transpõe de volta para (b, num_tokens, num_heads, head_dim)
        context_vec = context_vec.transpose(1, 2)

        # Concatena as cabeças em uma única dimensão: (b, num_tokens, d_out)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)

        # Projeção final (opcional) para misturar as saídas das cabeças
        context_vec = self.out_proj(context_vec)

        return context_vec


# Código para testar a atenção multi-cabeça
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2

# Cria uma instância da MultiHeadAttention com 2 cabeças
mha = MultiHeadAttention(d_in, d_out, context_length, dropout=0.0, num_heads=2)

# Aplica a atenção no batch de entrada
context_vecs = mha(batch)

# Exibe os vetores de contexto resultantes
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)  # Esperado: (2, 6, 2)


tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


In [40]:
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])
